In [1]:
import pandas as pd
import numpy as np
import requests
from lxml import html
from bs4 import BeautifulSoup

In [2]:
#r = requests.get('http://www.presidency.ucsb.edu/data.php')
#soup = BeautifulSoup(r.text, 'lxml')

In [3]:
#print(soup.prettify())
def trainMarkov(url, name):
    trumpmain = requests.get(url)
    trumpsoup=BeautifulSoup(trumpmain.text, 'lxml')
    root = 'http://www.presidency.ucsb.edu/'
    table = trumpsoup.find('table', width=700)
    links = []
    path=str("./"+ name +".txt")
    for z in table.findAll('a'):
        if('interview' not in z.text.lower()):
            links.append(root+(z['href'])[3:])

    trumpSpeeches = [requests.get(link , 'lxml')for link in links]
    trumpSpeechesSoup=[BeautifulSoup(speech.text, 'lxml') for speech in trumpSpeeches]
    

    with open(path, "a+", encoding='utf-8') as outFile:
        outFile.seek(0)
        for i,speech in enumerate(trumpSpeechesSoup):            
            outFile.write(trumpSpeechesSoup[i].find('span', class_='displaytext').text+'\n')
            
    # Get raw text as string.
    with open(path, encoding='utf-8') as f:
        text = f.read()

    # Build the model.
    text_model = markovify.Text(text)
    return text_model
            
    


In [4]:
#print(trumpSpeechesSoup[0].find('span', class_='displaytext').text)

In [5]:
#for table in soup.findAll('td', class_='doclist'):
    #print(table.prettify())

In [6]:
#'Donald-florida.txt'
import markovify



# Print five randomly-generated sentences
#for i in range(5):
#    print(text_model.make_sentence())

# Print three randomly-generated sentences of no more than 140 characters
#for i in range(3):
#    print(text_model.make_short_sentence(140))

In [7]:
hillarybot = trainMarkov('http://www.presidency.ucsb.edu/2016_election_speeches.php?candidate=70&campaign=2016CLINTON&doctype=5000', 'hillary')

In [8]:
berniebot = trainMarkov('http://www.presidency.ucsb.edu/2016_election_speeches.php?candidate=107&campaign=2016SANDERS&doctype=5000', 'bernie')

In [9]:
nixonbot = trainMarkov('http://www.presidency.ucsb.edu/1960_election_speeches.php?candidate=37', 'nixon')

In [10]:
trumpbot=trainMarkov('http://www.presidency.ucsb.edu/2016_election_speeches.php?candidate=45&campaign=2016TRUMP&doctype=5000', 'trump')

In [11]:
for i in range(20):
    print(trumpbot.make_short_sentence(max_chars=140))

Then she also wants to put an end to it on that same city block who just want to force the country under President Obama.
We will also be able to get going, OK?
Our country has ever done before.We will work.
You have 39 days to change it.There's some breaking news about that on November 1st, they're trying to get tired of not taking them on jobs.
We will terminate NAFTA and the Taliban in order to write this.
She was hiding her criminal conduct by having her chief-of-staff declare herself to be a fair trader.
That's why they've become vicious and vile.I will not lessen the danger, it will require military warfare, but also warm and fair-minded.
235 days, no news conference on that same city block who just want a lot going on.Her great disloyalty to the American Dream.
Then there was Russian money pouring into our poorest communities.Next, comes regulations.One of the other day?
Just get me all of those places?
So in theory it's supposed to be for sale, and important email records will 

In [12]:
model_combo = markovify.combine([ trumpbot, hillarybot ], [ 1, 1 ])

In [13]:
for i in range(20):
    print(model_combo.make_short_sentence(max_chars=140))

We've begun to change the fact that there are some issues that are making big investments in order to address this mounting mortgage crisis.
And, of course, all across this state.
As with so many who appreciate you and listening to folks here today from people who defend our values here at home.
That gives me no pause.
Now I know he took us out of love I'm talking about programs that hurt people.
And if your interested to see the last 6 years, on behalf of your President again.A vote for change, but work to improve their homes.
But Clinton is an honor.
Just like you sir, you sir, you sir, you and I do believe in universal health care they need.Our annual trade deficit with the primary.
No other country is founded on religious freedom.
Back in 1995, she was nine years old.
The soldiers from across the Middle East more unstable and dangerous globe.
But you can't be done with the Republican rule.
Since 9/11, hundreds of billions a year in America.
And perhaps we can make the first place.


In [14]:
import markovify
import nltk
import re

class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        words = re.split(self.word_split_pattern, sentence)
        words = [ "::".join(tag) for tag in nltk.pos_tag(words) ]
        return words

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

In [15]:
bettertrump = POSifiedText(trumpbot)

TypeError: expected string or bytes-like object